## 변동사항
### Doc2Vec 모델 수정
### Doc2Vec 모델 인풋 수정
### utf-8로 변환하는 것을 후방으로 뺌

In [4]:
import shutil, os, warnings, re, datetime, multiprocessing
warnings.filterwarnings(action='ignore')
import numpy as np
import pandas as pd
import tensorflow as tf
import _pickle as pickle
from functools import partial
import matplotlib.pyplot as plt
from gensim.models import Doc2Vec
from itertools import chain, combinations
from collections import namedtuple, Counter
from manipul import manipul_df as manipulation
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings(action='default')

In [5]:
def myDoc2Vec(data, d2v_size, model_loc, epoch=None, trainTF = False):
    """
    data : DataFrame([['DocID','Item']]) item : ['a','b',...,'z'] element in string type
    """
    data.columns = ['docID','item']
    data1 = [((row['item']), row['docID']) for idx, row in data.iterrows()]
    print(data1[0])
    data2 = namedtuple('TaggedDocument', 'words tags')
    tagged_data2 = [data2(d, [c]) for d, c in data1]
    
    if trainTF:
        model = Doc2Vec(
            dm = 0,  # 0 : PV-DBOW
            dbow_words = 0,  # 0 : train doc-vec only(faster)
            window = 8, vector_size = d2v_size, alpha = 0.025, min_alpha = 0.025, seed = 0, sample= 1e-5, 
            min_count=3, workers=multiprocessing.cpu_count(), hs = 0, negative = 10)
        model.build_vocab(tagged_data2)
        print('Doc2Vec training step--started')
        model.train(documents  = tagged_data2, total_examples = data.shape[0], epochs = epoch)
        model.save(model_loc)
        print('Doc2Vec training step--done. check--',model_loc)
    else:
        model=Doc2Vec.load(model_loc)
        print('Doc2Vec inference step--started')
        embedding_df = pd.DataFrame(
            data = [model.infer_vector(doc.words) for doc in tagged_data2], 
            columns = ["d2v"+str(i) for i in range(d2v_size)])
        print('Doc2Vec inference step--done')
        return embedding_df
    
    
def Listelement_str_to_byte(strlist, dim=2):
    if dim==2:
        result = []
        for ii in range(len(strlist)):
            tmp = []
            for jj in range(len(strlist[ii])):
                tmp.append(strlist[ii][jj].encode('utf-8'))
            result.append(tmp)
    elif dim==1:
        result = []
        for ii in range(len(strlist)):
            result.append(strlist[ii].encode('utf-8'))
    else:
        print('pz make custom function')
    return result

In [26]:
# 신규데이터 import
rawdata_loc = "./Datasets/Sourcefiles/181212/"

n주문 = pd.read_csv(rawdata_loc+"Order.csv", encoding = 'utf-8', dtype = 'str')
n주문상품 = pd.read_csv(rawdata_loc+"OrderProduct.csv", encoding = 'utf-8', dtype = 'str')
n주문상품출하 = pd.read_csv(rawdata_loc+"OrderProductShip.csv", encoding = 'utf-8', dtype = 'str')
n조마 = pd.read_csv(rawdata_loc+"info_Team.csv", encoding = 'utf-8', dtype = 'str')
n사업장 = pd.read_csv(rawdata_loc+"info_Bp.csv", encoding = 'utf-8', dtype = 'str')
n공유 = pd.read_csv(rawdata_loc+"info_ConstructionType.csv", encoding = 'utf-8', dtype = 'str')
n상마 = pd.read_csv(rawdata_loc+"info_Product.csv", encoding = 'utf-8', dtype = 'str')
n상품진열이력 = pd.read_csv(rawdata_loc+"hist_ProductView.csv", encoding = 'utf-8', dtype = 'str')
n상품공급사 = pd.read_csv(rawdata_loc+"info_ProductSupply.csv", encoding = 'utf-8', dtype = 'str')

# 기존데이터 Import
주문_2014 = pd.read_csv('./Datasets/Sourcefiles/Order_2014.csv', encoding = 'utf-8', dtype = 'str')
주문_2015 = pd.read_csv('./Datasets/Sourcefiles/Order_2015.csv', encoding = 'utf-8', dtype = 'str')
주문_2016 = pd.read_csv('./Datasets/Sourcefiles/Order_2016.csv', encoding = 'utf-8', dtype = 'str')
주문_2017 = pd.read_csv('./Datasets/Sourcefiles/Order_2017.csv', encoding = 'utf-8', dtype = 'str')
주문_2018 = pd.read_csv('./Datasets/Sourcefiles/Order_2018.csv', encoding = 'utf-8', dtype = 'str')
주문상품_2014 = pd.read_csv('./Datasets/Sourcefiles/OrderProduct_2014.csv', encoding = 'utf-8', dtype = 'str')
주문상품_2015 = pd.read_csv('./Datasets/Sourcefiles/OrderProduct_2015.csv', encoding = 'utf-8', dtype = 'str')
주문상품_2016 = pd.read_csv('./Datasets/Sourcefiles/OrderProduct_2016.csv', encoding = 'utf-8', dtype = 'str')
주문상품_2017 = pd.read_csv('./Datasets/Sourcefiles/OrderProduct_2017.csv', encoding = 'utf-8', dtype = 'str')
주문상품_2018 = pd.read_csv('./Datasets/Sourcefiles/OrderProduct_2018.csv', encoding = 'utf-8', dtype = 'str')
주문상품출하_2014 = pd.read_csv('./Datasets/Sourcefiles/OrderProductShip_2014.csv', encoding = 'utf-8', dtype = 'str')
주문상품출하_2015 = pd.read_csv('./Datasets/Sourcefiles/OrderProductShip_2015.csv', encoding = 'utf-8', dtype = 'str')
주문상품출하_2016 = pd.read_csv('./Datasets/Sourcefiles/OrderProductShip_2016.csv', encoding = 'utf-8', dtype = 'str')
주문상품출하_2017 = pd.read_csv('./Datasets/Sourcefiles/OrderProductShip_2017.csv', encoding = 'utf-8', dtype = 'str')
주문상품출하_2018 = pd.read_csv('./Datasets/Sourcefiles/OrderProductShip_2018.csv', encoding = 'utf-8', dtype = 'str')
조마=pd.read_csv('./Datasets/Sourcefiles/info_Team.csv',encoding ='utf-8',dtype = 'str')
사업장=pd.read_csv('./Datasets/Sourcefiles/info_Bp.csv',encoding ='utf-8',dtype = 'str')
공유=pd.read_csv('./Datasets/Sourcefiles/info_ConstructionType.csv',encoding ='utf-8',dtype = 'str')
상마=pd.read_csv('./Datasets/Sourcefiles/info_Product.csv',encoding ='utf-8',dtype = 'str')
상품공급사 = pd.read_csv('./Datasets/Sourcefiles/info_ProductSupply.csv',encoding='utf-8',dtype='str')
상품진열이력 = pd.read_csv('./Datasets/Sourcefiles/hist_item_open.csv', dtype = 'str') # 상품진열 히스토리

# 컬럼명 비교
print(
    sum(n주문.columns != 주문_2018.columns),
    sum(n주문상품.columns != 주문상품_2018.columns),
    sum(n주문상품출하.columns != 주문상품출하_2018.columns),
    
    sum(n조마.columns != 조마.columns),
    sum(n사업장.columns != 사업장.columns),
    sum(n공유.columns != 공유.columns),
    sum(n상마.columns != 상마.columns),
    sum(n상품공급사.columns != 상품공급사.columns),
    sum(n상품진열이력.columns != 상품진열이력.columns)
)

### info, hist류의 데이터는 신규 데이터로 업데이트 하고 주문 데이터는 날짜 별로 업데이트 한다.
조마 = n조마.copy()
사업장 = n사업장.copy()
공유 = n공유.copy()
상마 = n상마.copy()
상품공급사 = n상품공급사.copy()
상품진열이력 = n상품진열이력.copy()

경계일 = "2018-09-01" # 기존, 신규 사이 경계일
전일 = "2018-12-12" # 데이터 만료일+1 일

주문_2018 = 주문_2018[주문_2018.REGI_DATE_TIME<str(pd.to_datetime(경계일))]
n주문 = n주문[(
    (n주문.REGI_DATE_TIME>=str(pd.to_datetime(경계일)))&
    (n주문.REGI_DATE_TIME<str(pd.to_datetime(전일)))
)]

# Stage1 주문처리
주문 = pd.concat([주문_2014, 주문_2015, 주문_2016, 주문_2017, 주문_2018, n주문])
주문상품 = pd.concat([주문상품_2014, 주문상품_2015, 주문상품_2016, 주문상품_2017, 주문상품_2018, n주문상품])
주상출 = pd.concat([주문상품출하_2014, 주문상품출하_2015, 주문상품출하_2016, 주문상품출하_2017, 주문상품출하_2018,n주문상품출하])
print("ALL nesessary data imported")

0 0 0 0 0 0 0 0 0
ALL nesessary data imported


In [27]:
주문 = 주문[주문.GROUPID!='101'][[
    'ORDE_IDEN_NUMB', 'CONS_IDEN_NAME', 'GROUPID', 'CLIENTID', 'BRANCHID',
    'DELI_AREA_CODE','REGI_DATE_TIME','ORDE_USER_ID'
]].drop_duplicates(keep='first').reset_index(drop=True).fillna("")

주문상품 = 주문상품[[
    'ORDE_IDEN_NUMB', 'ORDE_SEQU_NUMB','GOOD_IDEN_NUMB',
    'VENDORID','ORDE_REQU_QUAN', 'DELI_MINI_DAY','REQU_DELI_DATE'
]].drop_duplicates(keep='first').reset_index(drop=True).dropna()

주상출 = 주상출[['ORDE_IDEN_NUMB','ORDE_SEQU_NUMB','DELI_STAT_FLAG',
           'DELI_DEGR_DATE']].drop_duplicates(keep='first').reset_index(drop=True)

주상출.ORDE_SEQU_NUMB = 주상출.ORDE_SEQU_NUMB.astype('int64')
주문상품.ORDE_SEQU_NUMB = 주문상품.ORDE_SEQU_NUMB.astype('int64')

주문주상 = pd.merge(주문,주문상품,on='ORDE_IDEN_NUMB',how='left')
주문주상출하 = pd.merge(주문주상,주상출,on=['ORDE_IDEN_NUMB','ORDE_SEQU_NUMB'],
                  how='left')

주문_total = 주문주상출하[주문주상출하.DELI_STAT_FLAG == '70'].drop(
    'DELI_STAT_FLAG', axis=1).reset_index(drop=True)

# Stage2 사업장 처리
사업장 = pd.concat([
        사업장[['BRANCHID','AREATYPE','BRANCHBUSITYPE','BRANCHBUSICLAS','WORKID']].rename(columns={'BRANCHID':'BORGID'}), 
        사업장[['BRANCHCD','AREATYPE','BRANCHBUSITYPE','BRANCHBUSICLAS','WORKID']].rename(columns={'BRANCHCD':'BORGID'})
    ], axis=0).drop_duplicates(keep='first')

공사유형_사업장 = pd.merge(
    사업장, 
    공유[['WORKID','WORKNM']].drop_duplicates(keep='first'), 
    how = 'left', on = 'WORKID')

조직마스터 = 조마[(조마.BORGTYPECD == 'BCH') & (조마.SVCTYPECD == 'BUY')]
조직마스터 = pd.concat([
        조직마스터[['BORGID','BORGNM']], 
        조직마스터[['BORGCD','BORGNM']].rename(columns={'BORGCD':'BORGID'})
    ], axis=0).drop_duplicates(keep='first')

조직_total = pd.merge(
    조직마스터, 공사유형_사업장, how = 'left', on = 'BORGID').rename(
    columns={'BORGID':'BRANCHID'}).fillna("")

주조_total = pd.merge(주문_total,조직_total,
              how='left',left_on='BRANCHID',right_on='BRANCHID').dropna()


상품마스터 = 상마[['good_iden_numb','good_name','good_spec',
                   'good_type','repre_good']].drop_duplicates(keep='first')

df_init = pd.merge(주조_total, 상품마스터, 
                   left_on = 'GOOD_IDEN_NUMB', right_on = 'good_iden_numb', 
                   how = 'left')
df_init = df_init[df_init.repre_good=="N"].reset_index(drop=True)

df_init = df_init.drop([
    "ORDE_SEQU_NUMB","GROUPID","ORDE_USER_ID","DELI_DEGR_DATE","DELI_MINI_DAY",
    "REQU_DELI_DATE","ORDE_REQU_QUAN","good_type","good_spec","good_name",
    "DELI_AREA_CODE","BORGNM","CONS_IDEN_NAME","repre_good","CLIENTID",
    "VENDORID","good_iden_numb","WORKNM"],
    axis=1)

df = df_init.rename(columns={
    "ORDE_IDEN_NUMB":"OrderNum", 
    "GOOD_IDEN_NUMB":"ProductCode",
    "BRANCHID":"BpID",
    "REGI_DATE_TIME":"OrderTime",
    "AREATYPE":"Region",
    "BRANCHBUSITYPE":"BpType",
    "BRANCHBUSICLAS":"BpClass",
    "WORKID":"ConstructionType"
})

In [68]:
df['OrderTime'] =  pd.to_datetime(df['OrderTime'])
df['OrderDate'] =  df['OrderTime'].dt.date
df = df.drop_duplicates(keep='first').reset_index(drop=True)

ActiveProduct = pd.read_csv(
    './Datasets/Sourcefiles/info_ProductSupply.csv',
    encoding='utf-8',
    dtype='str')#상품공급사
df['ProductActiveTF'] = df.ProductCode.isin(
    list(set(ActiveProduct[ActiveProduct.isUse=="1"].good_iden_numb)))

thres1 = Counter(df.ProductCode)
thres2 = {x : thres1[x] for x in thres1 if thres1[x]>10} # frequently purchased
thres3 = {x : thres1[x] for x in thres1 if thres1[x]<=10} # seldomly purchased

conditions = [
    (df['ProductCode'].isin(list(thres2.keys()))) & (df['ProductActiveTF']==True), # 자주 구매하면서 active
    (df['ProductCode'].isin(list(thres2.keys()))) & (df['ProductActiveTF']==False),# 자주 구매하면서 inactive ~99
    (df['ProductCode'].isin(list(thres3.keys())))# 덜 구매 ~98
]
choices = [df.ProductCode,"99999999999","99999999998"]
df['GrpProductCode'] = np.select(conditions,choices,default='none')

df1 = df.drop(['OrderNum','ProductActiveTF'],axis = 1)
df1 = df1.drop(["ProductCode"],axis=1).rename(columns={"GrpProductCode":"ProductCode"}) # 제품코드를 교체

(111280, 8)


In [78]:
with open('./Datasets/VocaList_181218_final.pkl','rb') as f:
    vL = pickle.load(f)
    
ListProductCode = [ii.decode('utf-8') for ii in vL['yv']]
# 기존에 있는 자재만 필터링
df1 = df1[df1.ProductCode.isin(ListProductCode)].reset_index(drop=True)

In [88]:
df1_1 = df1.groupby(['OrderDate','BpID','Region','ConstructionType','BpType','BpClass']).agg({
    "ProductCode":list,"OrderTime":"max"}).reset_index()
df2 = df1_1.sort_values(['BpID','OrderTime']).reset_index(drop = True)
print(df2.shape)

(108254, 8)


In [31]:
# Original
# # df = pd.read_csv('./Datasets/total_table_MainTotal.csv', dtype='str', encoding='utf-8')
# ActiveProduct = pd.read_csv('./Datasets/item_supply.csv',encoding='utf-8',dtype='str')#상품공급사
# ConstType = pd.read_csv('./Datasets/ConstructionType.csv',encoding='utf-8',dtype='str')#공사유형
# ConstType  = ConstType.rename(columns={"WORKNM":"ConstructionType"})
# ConstType = ConstType[["ConstructionType","WORKID"]].drop_duplicates()
# df = pd.merge(df, ConstType, on="ConstructionType", how="left").drop(
#     ["ConstructionType"],axis=1).rename(columns={"WORKID":"ConstructionType"})

# df = df[df.Option_rep=="N"].dropna()
# df['OrderTime'] =  pd.to_datetime(df['OrderTime'])
# df['OrderDate'] =  df['OrderTime'].dt.date
# df = df.drop(['SubOrderNum','GroupID','OrderAccount','ShipDate','DeliDate',
# 'RequestDeliDate','SupplyID','Supplynm','CompanyID','Companynm','Option_rep',
# 'Constructionnm','Bpnm','Region','Productnm','ProductSize','ProductType1',
# 'LCategory','MCategory','SCategory','OrderAmt','UnitPrice'], axis =1)
# df = df.rename(columns={'BpRegion':'Region'})
# df = df.drop_duplicates(keep='first')

# df['ProductActiveTF'] = df.ProductCode.isin(list(set(ActiveProduct[ActiveProduct.isUse=="1"].good_iden_numb)))

# thres1 = Counter(df.ProductCode)
# thres2 = {x : thres1[x] for x in thres1 if thres1[x]>10} # frequently purchased
# thres3 = {x : thres1[x] for x in thres1 if thres1[x]<=10} # seldomly purchased

# conditions = [
#     (df['ProductCode'].isin(list(thres2.keys()))) & (df['ProductActiveTF']==True), # 자주 구매하면서 active
#     (df['ProductCode'].isin(list(thres2.keys()))) & (df['ProductActiveTF']==False),# 자주 구매하면서 inactive ~99
#     (df['ProductCode'].isin(list(thres3.keys())))# 덜 구매 ~98
# ]
# choices = [df.ProductCode,"99999999999","99999999998"]
# df['GrpProductCode'] = np.select(conditions,choices,default='none')

# df1 = df.drop(['OrderNum','ProductActiveTF'],axis = 1)
# df1 = df1.drop(["ProductCode"],axis=1).rename(columns={"GrpProductCode":"ProductCode"}) # 제품코드를 교체
# df1 = df1.groupby(['OrderDate','BpID','Region','ConstructionType','BpType','BpClass']).agg({
#     "ProductCode":list,"OrderTime":"max"}).reset_index()
# df2 = df1.sort_values(['BpID','OrderTime']).reset_index(drop = True)
# print(df2.shape)
# Original end

In [89]:
df2['OrderYear'] =  df2.OrderTime.dt.year
loopindex = df2[['BpID', 'OrderYear']].drop_duplicates().sort_values(by=['BpID','OrderYear']).values
df2_1 = { 'BpID':[], 'OrderTime':[], 'x_ProductCodes':[],  'y_ProductCodes':[]}

for ii,jj in loopindex:
    looph1 = df2[(df2['BpID']==ii) & (df2["OrderYear"]==jj)].reset_index(drop=True)
    for kk in range(looph1.shape[0]):
        yy = looph1.iloc[kk+1:kk+2,:]
        xx = looph1.iloc[0:kk+1,:]
        if len(yy['ProductCode']) == 1:
            df2_1['x_ProductCodes'].append(list(set(chain.from_iterable(xx['ProductCode'].tolist()))))
            df2_1['y_ProductCodes'].append(list(set(chain.from_iterable(yy['ProductCode'].tolist()))))
            df2_1['OrderTime'].append(xx.OrderTime.max())
            df2_1['BpID'].append(ii)
            
df2_2 = pd.DataFrame(df2_1)

In [90]:
df2_2.shape # (108204, 4) ->

(105253, 4)

In [91]:
# Doc2Vec 임베딩 생성
start = datetime.datetime.now()
model_loc="./Trained_models/Models_Doc2Vec/Recommendation_15d2v_model_1207.model"
x_d2v_embed = myDoc2Vec(data=df2_2[['BpID','x_ProductCodes']], d2v_size=15,
                        trainTF=False, model_loc=model_loc)
x_d2v_embed2 = pd.DataFrame(pd.Series(x_d2v_embed.values.tolist()), columns=['d2v'])
duration = datetime.datetime.now()-start;m, s = divmod(duration.seconds, 60)
h, m = divmod(m, 60);print("[%02d:%02d:%02d]" %(h, m, s)) 

(['99999999999', '6601000067', '6601000010', '6601000053', '6601000174', '6601000024'], '304576')
Doc2Vec inference step--started
Doc2Vec inference step--done


In [92]:
# y아이템은 row단위로
y_ProductCodes = []
for ii in df2_2.y_ProductCodes.index:
    for jj in df2_2.y_ProductCodes[ii]:
        y_ProductCodes.append((ii, jj))
df3_2 = pd.DataFrame(y_ProductCodes, columns=['index','y_ProductCodes']).set_index('index')

# 전체 재통합
df3 = pd.merge(
    pd.concat([df2_2[["BpID","OrderTime","x_ProductCodes"]], x_d2v_embed2, df3_2], axis=1), 
    df2[['BpID','Region','ConstructionType','BpType','BpClass']].drop_duplicates(),
    on='BpID').reset_index(drop=True)

In [93]:
df3_3 = 상품진열이력

In [98]:
# 없는자재 제거
ListProductCode2 = [ii.decode('utf-8') for ii in vL['showv']]
df3_3 = df3_3[df3_3.GOOD_IDEN_NUMB.isin(ListProductCode2)].reset_index(drop=True)

In [99]:
# 노출 상품중 구매하지 않은 상품 리스트 추가하기. - Lookup table만들어 적용.
# df3_3 = pd.read_csv('./Datasets/hist_item_open.csv', dtype = 'str') # 상품진열 히스토리
df3_3 = df3_3[['GOOD_IDEN_NUMB','WORKINFO_ID_ARR','HIST_INSERT_DATE']].dropna()
df3_3['HIST_INSERT_DATE'] = pd.to_datetime(df3_3['HIST_INSERT_DATE'])

df3_3 = df3_3[df3_3.GOOD_IDEN_NUMB.isin(choices[0].tolist())].reset_index(drop=True)
df3_3['WorkList'] = df3_3['WORKINFO_ID_ARR'].str.split(',')

df3_4 =  df3_3.drop(['WORKINFO_ID_ARR'],axis=1).sort_values(
    by=['GOOD_IDEN_NUMB','HIST_INSERT_DATE']).reset_index(drop=True)
df3_4['HIST_FINISH_DATE'] = df3_4.HIST_INSERT_DATE.shift(-1)
df3_4['GOOD_IDEN_NUMB2'] = df3_4.GOOD_IDEN_NUMB.shift(-1)

df3_51 = df3_4[df3_4.GOOD_IDEN_NUMB == df3_4.GOOD_IDEN_NUMB2]
df3_52 = df3_4[df3_4.GOOD_IDEN_NUMB !=  df3_4.GOOD_IDEN_NUMB2]
df3_52 = df3_52.drop(["HIST_FINISH_DATE"],axis=1).insert(0,"HIST_FINISH_DATE",pd.datetime.now())

df3_6 = pd.concat([df3_51,df3_52]).sort_values(
    by=['GOOD_IDEN_NUMB','HIST_INSERT_DATE']).reset_index(drop=True)
df3_6 = df3_6[['GOOD_IDEN_NUMB','HIST_INSERT_DATE','HIST_FINISH_DATE','WorkList']]

df4 = pd.concat([
        pd.DataFrame(sum([list(zip([ii]*len(df3_6.WorkList[ii]),df3_6.WorkList[ii])) for ii in range(
                        df3_6.shape[0])],[])).set_index(0).rename(columns={1:'ConstructionType'}),
        df3_6[['GOOD_IDEN_NUMB','HIST_INSERT_DATE','HIST_FINISH_DATE']]], 
          axis =1)

df4_1 = df4.groupby('GOOD_IDEN_NUMB')['HIST_INSERT_DATE'].min().reset_index()
df4_2 = pd.merge(df4_1, df4, on=['GOOD_IDEN_NUMB','HIST_INSERT_DATE'], how='left')
init_ConstList = df4_2.groupby('ConstructionType')['GOOD_IDEN_NUMB'].apply(list).reset_index()

In [100]:
# 오래 걸리는 전처리 부분을 multiprocessing을 통해서 빠르게 처리
start = datetime.datetime.now()
num_cores = multiprocessing.cpu_count()-1#한개는 남겨둠.
num_partitions = num_cores
df_split = np.array_split(df3[['ConstructionType','OrderTime']], num_partitions)

if __name__ == '__main__':
    with multiprocessing.Pool(num_cores) as pool:
        func = partial(manipulation, df4, init_ConstList)
        mapped = pool.map(func, df_split)
        result = pd.DataFrame(pd.Series(list(chain.from_iterable(mapped))),columns =['Items'])
result.head()
duration = datetime.datetime.now()-start;m, s = divmod(duration.seconds, 60)
h, m = divmod(m, 60);print("[%02d:%02d:%02d]" %(h, m, s)) 

[00:04:55]


In [101]:
df4_5= pd.merge(df3,result,left_index=True,right_index=True)
df4_5['show_ProductCodes'] = pd.Series([list(set(df4_5['Items'][ii]) - set(df4_5['x_ProductCodes'][ii])) for ii in range(df4_5.shape[0])])
df5 = df4_5.drop(['Items'], axis = 1)

# x거래일자와 현재의 일자 차이 weight로 사용
daygap = (datetime.datetime.now() - df5.OrderTime).dt.days 
df5['daygap'] = 1-((daygap-min(daygap))/(max(daygap)-min(daygap))) # 0-1 normalization

df5 = df5[df5.y_ProductCodes !="99999999998"].reset_index(drop=True) # Y에서 덜 구매건은 아예 제외 함

##### item은 binary로 만들기.#### 추가
df5['x_ProductCodes'] = pd.Series(Listelement_str_to_byte(df5.x_ProductCodes.tolist(), dim=2))
df5['y_ProductCodes'] = pd.Series(Listelement_str_to_byte(df5.y_ProductCodes.tolist(), dim=1))
df5['show_ProductCodes'] = pd.Series(Listelement_str_to_byte(df5.show_ProductCodes.tolist(), dim=2))

In [102]:
print(df5.shape) #(337388, 11)
df5.head(3)

(335494, 11)


,BpID,OrderTime,x_ProductCodes,d2v,y_ProductCodes,Region,ConstructionType,BpType,BpClass,show_ProductCodes,daygap
0,304576,2014-03-05 10:49:56.520,"[b'99999999999', b'6601000067', b'6601000010',...","[-0.2364179939031601, -1.1428827047348022, 1.8...",b'6601000055',12,7,정보통신서비스,서비스,"[b'2900900001', b'7000200010', b'6601000076', ...",0.034444
1,304576,2014-03-05 10:49:56.520,"[b'99999999999', b'6601000067', b'6601000010',...","[-0.2364179939031601, -1.1428827047348022, 1.8...",b'99999999999',12,7,정보통신서비스,서비스,"[b'2900900001', b'7000200010', b'6601000076', ...",0.034444
2,304576,2014-03-05 10:49:56.520,"[b'99999999999', b'6601000067', b'6601000010',...","[-0.2364179939031601, -1.1428827047348022, 1.8...",b'6601000067',12,7,정보통신서비스,서비스,"[b'2900900001', b'7000200010', b'6601000076', ...",0.034444


In [6]:
with open('./Datasets/VocaList_181218_final.pkl','rb') as f:
    vL = pickle.load(f)

In [9]:
len(vL['yv'])

2324

In [11]:
# vL = {"xv":list(set(chain.from_iterable(df5.x_ProductCodes))-{b"99999999998"}),
#       "showv":list(set(chain.from_iterable(df5.show_ProductCodes))), # show에서는 빼지 않는다. (취지와도 안맞고 빼면 []이 됨.)
#       "yv":tuple(df5.y_ProductCodes.unique())}

# print(np.array(sorted(vL["xv"]))) # 99999999998이 없어야 함.

# with open('./Datasets/df5_tmp.pkl','wb') as f:
#     pickle.dump(df5, f)
# with open('./Datasets/VocaList.pkl','wb') as f:
#     pickle.dump(vL, f)

# with open('./Datasets/VocaList.pkl','rb') as f:
#     vL = pickle.load(f)

[b'10000000052' b'10000000053' b'10000000055' ... b'7400900176'
 b'7400900179' b'99999999999']


In [103]:
def make_Example(row):
    example= tf.train.Example(features=tf.train.Features(feature={
        'Region':tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['Region'].encode('utf-8')])),
        'BpType':tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['BpType'].encode('utf-8')])),
        'BpClass':tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['BpClass'].encode('utf-8')])),
        'ConstructionType':tf.train.Feature(bytes_list=tf.train.BytesList(
            value=[row['ConstructionType'].encode('utf-8')])),
        'd2v':tf.train.Feature(float_list=tf.train.FloatList(value=row['d2v'])),
        'daygap':tf.train.Feature(float_list=tf.train.FloatList(value=[row['daygap']])),
        'show_ProductCodes':tf.train.Feature(bytes_list=tf.train.BytesList(value= row['show_ProductCodes'])),
        'x_ProductCodes':tf.train.Feature(bytes_list=tf.train.BytesList(value= row['x_ProductCodes'])),
        'y_ProductCodes':tf.train.Feature(bytes_list=tf.train.BytesList(value=[row['y_ProductCodes']]))
    }))
    return example

def create_tfrecords_file(tf_filename= 'data.tfrecord', input_df=None):
    writer = tf.python_io.TFRecordWriter(tf_filename)
    print("Creating TFRecords file at", tf_filename, "...")
    def mk_tfrecords(row):
        example = make_Example(row)
        content = example.SerializeToString()
        writer.write(content)
        return example
    input_df.apply(mk_tfrecords, axis=1)
    writer.close()
    print("Finish Writing", tf_filename)

In [104]:
np.random.seed(seed=0)
sampleidx = np.random.choice(df5[df5['OrderTime']>="2018-06-01"].index,size=10000,replace=False) # size 변경 필.

conditions = [(df5['OrderTime']>='2018-06-01') & (~df5.index.isin(sampleidx)),
              (df5['OrderTime']<"2018-06-01") | ((df5['OrderTime']>="2018-06-01") & (df5.index.isin(sampleidx)))]
choices = ['eval','train']

df5['usage'] = np.select(conditions,choices,default='none')
print(Counter(df5['usage']))

for etp in df5['usage'].unique():
    tmp_df = df5[df5['usage']==etp]
    tmp_p_dist = tmp_df.y_ProductCodes.value_counts()/tmp_df.shape[0]
    print("Entropy of %s set : [%.3f]"%(etp, -np.log2(tmp_p_dist).mean()))

for ii in df5.usage.unique():
    tmp_df = df5[df5.usage==ii].drop(['BpID','OrderTime','usage'],axis=1)
    filename = './Datasets/'+ii+'_181218_final.tfrecord'
    create_tfrecords_file(filename, tmp_df)

Counter({'train': 300264, 'eval': 35230})
Entropy of train set : [12.690]
Entropy of eval set : [12.381]
Creating TFRecords file at ./Datasets/train_181218_final.tfrecord ...
Finish Writing ./Datasets/train_181218_final.tfrecord
Creating TFRecords file at ./Datasets/eval_181218_final.tfrecord ...
Finish Writing ./Datasets/eval_181218_final.tfrecord


In [105]:
# with open('./Datasets/test_xwd_d2v.pkl','wb') as f:
#     pickle.dump(df5[df5.usage=='test'], f)
with open('./Datasets/eval_181218_final.pkl','wb') as f:
    pickle.dump(df5[df5.usage=='eval'], f)

In [10]:
read_f = {
    'Region': tf.FixedLenFeature([], dtype=tf.string),
    'BpType': tf.FixedLenFeature([], dtype=tf.string),
    'BpClass': tf.FixedLenFeature([], dtype=tf.string),
    'ConstructionType': tf.FixedLenFeature([], dtype=tf.string),
    'd2v':tf.FixedLenFeature([15], dtype=tf.float32),
    'daygap':tf.FixedLenFeature([], dtype=tf.float32),
    'show_ProductCodes': tf.VarLenFeature(dtype=tf.string),
    'x_ProductCodes': tf.VarLenFeature(dtype=tf.string),
    'y_ProductCodes': tf.FixedLenFeature([], dtype=tf.string)
}

def input_fn(filename, predict=False):
    def parser(serialized_exp):
        columns = tf.parse_single_example(serialized=serialized_exp,features=read_f)
        label = columns['y_ProductCodes']
        del columns['y_ProductCodes']
        features = columns
        return features, label
    dataset= tf.data.TFRecordDataset(filenames=filename)
    if predict == False:
        dataset = dataset.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=100,count=-1))
    dataset = dataset.apply(tf.data.experimental.map_and_batch(
            map_func=parser, batch_size=64, num_parallel_batches = 16))
    # BP:number of elements / batches consumed by a training step
    dataset = dataset.cache().prefetch(buffer_size = 50) 
    return dataset

In [11]:
def train_input_fn():
    return input_fn(filename='./Datasets/train_181218_final.tfrecord')

def eval_input_fn():
    return input_fn(filename='./Datasets/eval_181218_final.tfrecord')

def pred_input_fn():
    return input_fn(filename='./Datasets/eval_181218_final.tfrecord', predict=True)

In [12]:
BpClass=tf.feature_column.categorical_column_with_hash_bucket("BpClass", hash_bucket_size=300)
BpType=tf.feature_column.categorical_column_with_hash_bucket("BpType", hash_bucket_size=300)
ConstructionType=tf.feature_column.categorical_column_with_hash_bucket(
    "ConstructionType", hash_bucket_size=100)
Region=tf.feature_column.categorical_column_with_hash_bucket("Region", hash_bucket_size=100)
d2v = tf.feature_column.numeric_column('d2v',shape=[15], dtype=tf.float64)
daygap = tf.feature_column.numeric_column('daygap',dtype=tf.float64)
X_item = tf.feature_column.categorical_column_with_vocabulary_list(
    key="x_ProductCodes",vocabulary_list=vL["xv"])
Show_item = tf.feature_column.categorical_column_with_vocabulary_list(
    key="show_ProductCodes",vocabulary_list=vL["showv"])

wide_columns = [
    X_item,
    Show_item,
    ConstructionType,
    tf.feature_column.crossed_column(["ConstructionType", "Region"], hash_bucket_size=int(1e3))
]
deep_columns = [
    d2v,
    tf.feature_column.embedding_column(BpClass, dimension=16), 
    tf.feature_column.embedding_column(BpType, dimension=16), 
    tf.feature_column.embedding_column(ConstructionType, dimension=16), 
    tf.feature_column.embedding_column(Region, dimension=16)
]

In [13]:
_config = tf.estimator.RunConfig(
    save_checkpoints_secs=None,save_checkpoints_steps=1000,keep_checkpoint_max=2)

tot_model = tf.estimator.DNNLinearCombinedClassifier(
    model_dir="./Trained_models/Models_WnD/xwd2/",
    linear_feature_columns=wide_columns,
    dnn_feature_columns=deep_columns,
    dnn_hidden_units=[2048,512,128],
    dnn_dropout = 0.3,
    n_classes = len(vL["yv"]),
    label_vocabulary= vL["yv"],
    weight_column = daygap,
    batch_norm = True,
    warm_start_from="./Trained_models/Models_WnD/xwd2/",
    config = _config)

INFO:tensorflow:Using config: {'_num_worker_replicas': 1, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fdbfaabcc50>, '_tf_random_seed': None, '_eval_distribute': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_save_summary_steps': 100, '_service': None, '_evaluation_master': '', '_task_type': 'worker', '_experimental_distribute': None, '_device_fn': None, '_keep_checkpoint_max': 2, '_protocol': None, '_global_id_in_cluster': 0, '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_train_distribute': None, '_num_ps_replicas': 0, '_task_id': 0, '_save_checkpoints_secs': None, '_master': '', '_model_dir': './Trained_models/Models_WnD/xwd2/', '_save_checkpoints_steps': 1000, '_log_step_count_steps': 100}


In [14]:
# # train step # = trains데이터수/64=4500 -> 10,000=2epoch
# # eval step # = eval데이터수/64=240 -> 500=2epoch
# tot_model.train(input_fn=train_input_fn, steps = int(1e4)) 
# tot_model.evaluate(input_fn=eval_input_fn,steps=500) 
while True:
    tot_model.train(input_fn=train_input_fn, steps = int(1e4)) 
    tot_model.evaluate(input_fn=eval_input_fn,steps=500) 

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.5/dist-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  return _inspect.getargspec(target)


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='./Trained_models/Models_WnD/xwd2/', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: ('./Trained_models/Models_WnD/xwd2/',)
INFO:tensorflow:Warm-starting variable: dnn/hiddenlayer_1/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: linear/linear_model/ConstructionType/weights; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dnn/hiddenlayer_0/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dnn/input_from_feature_columns/input_layer/BpType_embedding/embedding_weights; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: linear/linear_model/x_ProductCodes/weights; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dnn/hiddenlayer_2/batchnorm_2/beta; prev_var_name: Unchanged
INFO:tensorflow:Warm-start

INFO:tensorflow:global_step/sec: 8.37939
INFO:tensorflow:loss = 36.34574, step = 475701 (11.933 sec)
INFO:tensorflow:global_step/sec: 9.51285
INFO:tensorflow:loss = 155.42328, step = 475801 (10.512 sec)
INFO:tensorflow:global_step/sec: 9.44494
INFO:tensorflow:loss = 94.04799, step = 475901 (10.589 sec)
INFO:tensorflow:Saving checkpoints for 476000 into ./Trained_models/Models_WnD/xwd2/model.ckpt.
INFO:tensorflow:global_step/sec: 6.84199
INFO:tensorflow:loss = 152.21445, step = 476001 (14.610 sec)
INFO:tensorflow:global_step/sec: 7.98139
INFO:tensorflow:loss = 161.14508, step = 476101 (12.542 sec)
INFO:tensorflow:global_step/sec: 8.4843
INFO:tensorflow:loss = 80.31291, step = 476201 (11.782 sec)
INFO:tensorflow:global_step/sec: 9.77145
INFO:tensorflow:loss = 212.03044, step = 476301 (10.227 sec)
INFO:tensorflow:global_step/sec: 8.90235
INFO:tensorflow:loss = 129.65103, step = 476401 (11.236 sec)
INFO:tensorflow:global_step/sec: 12.0773
INFO:tensorflow:loss = 177.96786, step = 476501 (8.

INFO:tensorflow:Warm-starting variable: dnn/hiddenlayer_2/batchnorm_2/gamma; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: linear/linear_model/ConstructionType_X_Region/weights; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dnn/hiddenlayer_1/batchnorm_1/gamma; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: linear/linear_model/bias_weights; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dnn/input_from_feature_columns/input_layer/ConstructionType_embedding/embedding_weights; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dnn/hiddenlayer_0/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dnn/hiddenlayer_1/bias; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dnn/hiddenlayer_0/batchnorm_0/gamma; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: dnn/logits/kernel; prev_var_name: Unchanged
INFO:tensorflow:Warm-starting variable: linear/linear_mod

...

INFO:tensorflow:global_step/sec: 8.23331
INFO:tensorflow:loss = 93.58241, step = 497801 (12.145 sec)
INFO:tensorflow:global_step/sec: 8.29093
INFO:tensorflow:loss = 198.85219, step = 497901 (12.061 sec)
INFO:tensorflow:Saving checkpoints for 498000 into ./Trained_models/Models_WnD/xwd2/model.ckpt.
INFO:tensorflow:global_step/sec: 7.19366
INFO:tensorflow:loss = 83.065475, step = 498001 (13.896 sec)
INFO:tensorflow:global_step/sec: 9.06512
INFO:tensorflow:loss = 189.79453, step = 498101 (11.035 sec)
INFO:tensorflow:global_step/sec: 10.2606
INFO:tensorflow:loss = 123.04342, step = 498201 (9.754 sec)
INFO:tensorflow:global_step/sec: 10.235
INFO:tensorflow:loss = 120.75604, step = 498301 (9.767 sec)
INFO:tensorflow:global_step/sec: 10.6089
INFO:tensorflow:loss = 222.1394, step = 498401 (9.423 sec)
INFO:tensorflow:global_step/sec: 9.74817
INFO:tensorflow:loss = 83.89989, step = 498501 (10.266 sec)
INFO:tensorflow:global_step/sec: 8.0445
INFO:tensorflow:loss = 131.24237, step = 498601 (12.422

INFO:tensorflow:global_step/sec: 8.28542
INFO:tensorflow:loss = 72.55939, step = 501401 (12.070 sec)
INFO:tensorflow:global_step/sec: 8.77739
INFO:tensorflow:loss = 110.806244, step = 501501 (11.390 sec)
INFO:tensorflow:global_step/sec: 9.73742
INFO:tensorflow:loss = 150.6163, step = 501601 (10.268 sec)
INFO:tensorflow:global_step/sec: 9.38992
INFO:tensorflow:loss = 89.507706, step = 501701 (10.654 sec)
INFO:tensorflow:global_step/sec: 12.0486
INFO:tensorflow:loss = 173.87512, step = 501801 (8.295 sec)
INFO:tensorflow:global_step/sec: 13.8664
INFO:tensorflow:loss = 223.02852, step = 501901 (7.213 sec)
INFO:tensorflow:Saving checkpoints for 502000 into ./Trained_models/Models_WnD/xwd2/model.ckpt.
INFO:tensorflow:global_step/sec: 8.79338
INFO:tensorflow:loss = 115.85833, step = 502001 (11.371 sec)
INFO:tensorflow:global_step/sec: 21.5998
INFO:tensorflow:loss = 163.80069, step = 502101 (4.631 sec)
INFO:tensorflow:global_step/sec: 18.4246
INFO:tensorflow:loss = 194.97575, step = 502201 (5.

INFO:tensorflow:loss = 91.11488, step = 508801 (11.258 sec)
INFO:tensorflow:global_step/sec: 9.33307
INFO:tensorflow:loss = 49.89338, step = 508901 (10.703 sec)
INFO:tensorflow:Saving checkpoints for 509000 into ./Trained_models/Models_WnD/xwd2/model.ckpt.
INFO:tensorflow:global_step/sec: 9.44404
INFO:tensorflow:loss = 77.69331, step = 509001 (10.586 sec)
INFO:tensorflow:global_step/sec: 9.43567
INFO:tensorflow:loss = 71.5511, step = 509101 (10.602 sec)
INFO:tensorflow:global_step/sec: 8.22607
INFO:tensorflow:loss = 118.92496, step = 509201 (12.157 sec)
INFO:tensorflow:global_step/sec: 9.35138
INFO:tensorflow:loss = 208.34885, step = 509301 (10.697 sec)
INFO:tensorflow:global_step/sec: 11.0863
INFO:tensorflow:loss = 76.02121, step = 509401 (9.012 sec)
INFO:tensorflow:global_step/sec: 8.38389
INFO:tensorflow:loss = 106.89839, step = 509501 (11.933 sec)
INFO:tensorflow:global_step/sec: 9.25248
INFO:tensorflow:loss = 52.936962, step = 509601 (10.810 sec)
INFO:tensorflow:global_step/sec: 9

INFO:tensorflow:loss = 189.53143, step = 512401 (5.861 sec)
INFO:tensorflow:global_step/sec: 16.4693
INFO:tensorflow:loss = 160.46295, step = 512501 (6.074 sec)
INFO:tensorflow:global_step/sec: 15.0319
INFO:tensorflow:loss = 126.23038, step = 512601 (6.655 sec)
INFO:tensorflow:global_step/sec: 7.91827
INFO:tensorflow:loss = 239.89944, step = 512701 (12.631 sec)
INFO:tensorflow:global_step/sec: 8.2105
INFO:tensorflow:loss = 119.658966, step = 512801 (12.177 sec)
INFO:tensorflow:global_step/sec: 7.8263
INFO:tensorflow:loss = 146.0073, step = 512901 (12.777 sec)
INFO:tensorflow:Saving checkpoints for 513000 into ./Trained_models/Models_WnD/xwd2/model.ckpt.
INFO:tensorflow:global_step/sec: 6.41903
INFO:tensorflow:loss = 123.206314, step = 513001 (15.576 sec)
INFO:tensorflow:global_step/sec: 8.52727
INFO:tensorflow:loss = 133.26918, step = 513101 (11.734 sec)
INFO:tensorflow:global_step/sec: 7.97907
INFO:tensorflow:loss = 160.47958, step = 513201 (12.543 sec)
INFO:tensorflow:global_step/sec

KeyboardInterrupt: 

# 평가평가

In [15]:
# 평가 with predict data
with open('./Datasets/eval_181218_final.pkl','rb') as f:
    test1 = pickle.load(f)

true_label = test1.y_ProductCodes.tolist()
y_voca = np.array(vL["yv"])
ex_item = y_voca != b"99999999999"

p_result = tot_model.predict(input_fn=pred_input_fn)
answers = []
  
for num, item in enumerate(p_result):
    if num>=len(true_label):
        break
    else:
        prob_list = ex_item*item['probabilities']
        esti_label = y_voca[np.argsort(np.argsort(prob_list)[::-1])<5]
        answers.append(int(true_label[num] in esti_label))
print(np.mean(answers))

p_result = tot_model.predict(input_fn=pred_input_fn)
answers = []
  
for num, item in enumerate(p_result):
    if num>=len(true_label):
        break
    else:
        prob_list = ex_item*item['probabilities']
        esti_label = y_voca[np.argsort(np.argsort(prob_list)[::-1])<10]
        answers.append(int(true_label[num] in esti_label))
print(np.mean(answers))

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.5/dist-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  return _inspect.getargspec(target)


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./Trained_models/Models_WnD/xwd2/model.ckpt-516000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
0.23732614249219416
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./Trained_models/Models_WnD/xwd2/model.ckpt-516000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
0.3682940675560602


In [134]:
import os.path, time
folder = "../skts_project/Datasets/"
foldertime = time.ctime(os.path.getmtime(file))
foldertime2 = datetime.datetime.strptime(foldertime, "%a %b %d %H:%M:%S %Y")
foldertime2
# 로드 피클
# 마지막 날짜와 비교
# 현 날짜가 피클 마지막 보다 더 늦은 시간이면 변경
# 현 날짜 피클에 추가
# 피클 저장

datetime.datetime(2018, 12, 18, 7, 8, 8)

In [ ]:
주문_2014 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문_2014.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
주문_2015 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문_2015.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
주문_2016 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문_2016.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
주문_2017 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문_2017.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
주문_2018 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문_2018.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
주문상품_2014 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문상품_2014.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
주문상품_2015 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문상품_2015.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
주문상품_2016 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문상품_2016.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
주문상품_2017 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문상품_2017.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
주문상품_2018 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문상품_2018.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
주문상품출하_2014 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문상품출하_2014.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
주문상품출하_2015 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문상품출하_2015.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
주문상품출하_2016 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문상품출하_2016.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
주문상품출하_2017 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문상품출하_2017.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
주문상품출하_2018 = pd.read_csv('C:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가5차_180920/주문상품출하_2018.csv', encoding = 'utf-8', index_col=0, dtype = 'str')
상품마스터 = pd.read_csv('c:/Users/kim85/OneDrive/Project_AgileSoda/원본/추가6차_181005/상품마스터.csv',encoding ='utf-8',dtype = 'str')
조직마스터 = pd.read_csv('c:/users/kim85/OneDrive/Project_AgileSoda/원본/추가6차_181005/조직마스터.csv', encoding ='utf-8', dtype ='str')
사업장= pd.read_csv('c:/users/kim85/OneDrive/Project_AgileSoda/원본/추가6차_181005/사업장.csv', encoding ='utf-8', dtype ='str')
공사유형 = pd.read_csv('c:/users/kim85/OneDrive/Project_AgileSoda/원본/추가6차_181005/공사유형.csv', encoding ='utf-8', dtype ='str')

주문 = pd.concat([주문_2014, 주문_2015, 주문_2016, 주문_2017, 주문_2018])
주문상품 = pd.concat([주문상품_2014, 주문상품_2015, 주문상품_2016, 주문상품_2017, 주문상품_2018])
주문상품출하 = pd.concat([주문상품출하_2014, 주문상품출하_2015, 주문상품출하_2016, 주문상품출하_2017, 주문상품출하_2018])

# 주문 처리
주문 = 주문.reset_index()[['ORDE_IDEN_NUMB','CONS_IDEN_NAME','GROUPID','CLIENTID','BRANCHID',
                       'DELI_AREA_CODE','REGI_DATE_TIME','ORDE_USER_ID']].drop_duplicates(keep='first')
주문상품 = 주문상품.reset_index()[['ORDE_IDEN_NUMB','ORDE_SEQU_NUMB','GOOD_IDEN_NUMB',
                           'ORDE_REQU_QUAN']].drop_duplicates(keep='first')
주문상품['ORDE_REQU_QUAN'] = 주문상품['ORDE_REQU_QUAN'].astype('int64')
주문상품 = 주문상품.groupby(['ORDE_IDEN_NUMB','GOOD_IDEN_NUMB'])['ORDE_REQU_QUAN'].agg('sum').reset_index()
주문상품출하 =  주문상품출하.reset_index()[['ORDE_IDEN_NUMB', 'DELI_STAT_FLAG']].drop_duplicates(keep='first')
주문상품출하 = 주문상품출하[주문상품출하.DELI_STAT_FLAG=='70'].ORDE_IDEN_NUMB.unique()

주문상품_주문 = pd.merge(주문상품, 주문, on = 'ORDE_IDEN_NUMB', how = 'left')
주문상품_주문_출하 = 주문상품_주문[주문상품_주문.ORDE_IDEN_NUMB.isin(주문상품출하)]
주문전체 = 주문상품_주문_출하[주문상품_주문_출하.GROUPID != '101'].drop_duplicates(keep='first')

# 상품 처리
상품마스터 = 상품마스터[['good_iden_numb','cate_id','good_name','good_spec','good_type','repre_good']].drop_duplicates(keep='first')
상품마스터 = 상품마스터.rename(columns = {'good_iden_numb':'GOOD_IDEN_NUMB'})
# repre_good - Y : 옵션대표상품, N : 단품, P : 옵션상품 // good_type - 10 : 일반, 20 : 지정,60 : 공구, 70 : 안전,80 : 보안

상품주문전체 = pd.merge(주문전체, 상품마스터, on = 'GOOD_IDEN_NUMB', how = 'left')
상품주문전체 = 상품주문전체[상품주문전체.repre_good=='N'] # 옵션상품 제외

In [ ]:
# 사업장 처리
사업장 = pd.concat([
        사업장[['BRANCHID','AREATYPE','BRANCHBUSITYPE','BRANCHBUSICLAS','WORKID']].rename(columns={'BRANCHID':'BORGID'}), 
        사업장[['BRANCHCD','AREATYPE','BRANCHBUSITYPE','BRANCHBUSICLAS','WORKID']].rename(columns={'BRANCHCD':'BORGID'})
    ], axis=0).drop_duplicates(keep='first')

공사유형_사업장 = pd.merge(
    사업장, 
    공사유형[['WORKID','WORKNM']].drop_duplicates(keep='first'), 
    how = 'left', on = 'WORKID')

조직마스터 = 조직마스터[(조직마스터.BORGTYPECD == 'BCH') & (조직마스터.SVCTYPECD == 'BUY')] # 사업장레벨 및 구매사만
조직마스터 = pd.concat([
        조직마스터[['BORGID','BORGNM']], 
        조직마스터[['BORGCD','BORGNM']].rename(columns={'BORGCD':'BORGID'})
    ], axis=0).drop_duplicates(keep='first')

조직전체 = pd.merge(조직마스터, 공사유형_사업장, how = 'left', on = 'BORGID').rename(columns={'BORGID':'BRANCHID'})
